In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
import torch.nn as nn
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv('/kaggle/input/lish-moa/train_features.csv')
target = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')
del data['sig_id']

categorical_cols = ['cp_type', 'cp_dose']
categorical_features = data[categorical_cols]
data.drop(categorical_cols, axis=1, inplace=True)
categorical_features = OneHotEncoder(sparse=False).fit_transform(categorical_features)
data = StandardScaler().fit_transform(data)

X = np.concatenate((categorical_features, data), axis=1)
y = target.to_numpy()[:,1:].astype(float)

In [ ]:
def k_folds_gen(*arrs, test_size=0.1):
    indeces = np.random.permutation(arrs[0].shape[0])
    
    batch_size = int(arrs[0].shape[0] * 0.1)
    for start in range(0, len(indeces), batch_size):
        indx_val = indeces[start: start+batch_size]
        indx_train = np.concatenate((indeces[:start], indeces[start+batch_size:]), axis=0)
        
        to_return = []
        for arr in arrs:
            to_return.append(arr[indx_train])
            to_return.append(arr[indx_val])
        yield tuple(to_return)

In [ ]:
def batch_iterator(*arrs, batch_size):
    indeces = np.random.permutation(arrs[0].shape[0])
    
    
    for start in range(0, len(indeces), batch_size):
        indx = indeces[start: start+batch_size]
        to_return = []
        for arr in arrs:
            to_return.append(arr[indx])
        yield tuple(to_return)

In [ ]:
def create_N_models(N, device):
    models = [None] * N
    for i in range(len(models)):
        models[i] = nn.Sequential(
            nn.Linear(877, 256),
            nn.BatchNorm1d(256),
            nn.Dropout(p=0.8),
            nn.LeakyReLU(),
            nn.Linear(256, 206),
            nn.Sigmoid(),
        )
        models[i].to(device)
    return models

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
models = create_N_models(10, device)
for i in range(len(models)):
    models[i].load_state_dict(torch.load(f"/kaggle/input/k-folds-weights/model_{i}.pth", map_location=device))
batch_size=128

In [ ]:
# n = 3
# prob = 10 / 206
# criterion = nn.BCELoss()
# k_fold_generator = iter(k_folds_gen(X, y, test_size=0.1))

# for model in models:
#     opt = torch.optim.Adam(model.parameters())
    
#     final_loss = []
#     final_val_loss = []
#     final_true_loss = []
    
#     X_train, X_val, y_train, y_val = next(k_fold_generator)
    
#     for i in range(250):
#         model.train()
#         losses = []
#         true_losses = []

#         for X_batch, y_batch in batch_iterator(X_train, y_train, batch_size=batch_size):

#             X_batch = torch.tensor(X_batch, dtype=torch.float32).to(device)

#             true_y_batch = torch.tensor(y_batch, dtype=torch.float32).to(device)
#             ind = np.random.randint(0, y_batch.shape[0], size=n)
#             y_batch[ind] = np.ones_like(y_batch[ind]) * (np.random.rand(n, 206) < prob)
#             y_batch = torch.tensor(y_batch, dtype=torch.float32).to(device)

#             p = model(X_batch)
#             true_loss = criterion(p, true_y_batch)
#             loss = criterion(p, y_batch)
#             total_loss = loss + 0.0001*sum(map(lambda x: torch.sum(x**2), list(model.parameters())[-1:]))
#             total_loss.backward()
#             opt.step()
#             opt.zero_grad()
#             losses.append(loss.cpu().detach().numpy())
#             true_losses.append(true_loss.cpu().detach().numpy())

#         for g in opt.param_groups:
#             g['lr'] = g['lr']*0.99

#         loss = np.mean(losses)
#         final_loss.append(loss)
#         final_true_loss.append(np.mean(true_losses))
#         losses = []
#         model.eval()
#         with torch.no_grad():
#             for X_batch, y_batch in batch_iterator(X_val, y_val, batch_size=batch_size):

#                 X_batch = torch.tensor(X_batch, dtype=torch.float32).to(device)
#                 y_batch = torch.tensor(y_batch, dtype=torch.float32).to(device)

#                 p = model(X_batch)           
#                 loss = criterion(p, y_batch)
#                 losses.append(loss.cpu().detach().numpy())
#         val_loss = np.mean(losses)
#         final_val_loss.append(val_loss)
        
#         if (i+1) % 10 == 0:
#             print('Iteration {}'.format(i+1))
#             print('  {:.5f} loss'.format(final_loss[-1]))
#             print('  {:.5f} true loss'.format(final_true_loss[-1]))
#             print('  {:.5f} val loss'.format(val_loss))
#         if final_true_loss[-1] < 0.015 and val_loss < 0.015:
#             break

#     plt.plot(final_loss, label='train_loss')
#     plt.plot(final_val_loss, label='val_loss')
#     plt.plot(final_true_loss, label='true_train_loss')
#     plt.legend()
#     plt.ylim(0, 0.020)
#     plt.grid()
#     plt.xticks(range(0, epoch+1, 10))
#     plt.show()

In [ ]:
test_data = pd.read_csv('/kaggle/input/lish-moa/test_features.csv')
sig_id = test_data['sig_id']
del test_data['sig_id']

categorical_features = test_data[categorical_cols]
test_data.drop(categorical_cols, axis=1, inplace=True)
categorical_features = OneHotEncoder(sparse=False).fit_transform(categorical_features)
test_data = StandardScaler().fit_transform(test_data)
X_test = np.concatenate((categorical_features, test_data), axis=1)

In [ ]:
for i in range(len(models)):
    models[i].eval()
    
with torch.no_grad():
    predictions = (sum(map(lambda x: x(torch.tensor(X_test, dtype=torch.float32).to(device)), models)) /
                   len(models)).to('cpu').detach().numpy()
predictions.shape

In [ ]:
submission = pd.DataFrame(predictions)
submission.columns = target.columns[1:]
submission['sig_id'] = sig_id
submission.head()

In [ ]:
submission.to_csv('submission.csv', index=False)